In [238]:
from uuid import uuid4

from pdb2sql import pdb2sql
import numpy
import pandas as pd

from deeprankcore.models.structure import Chain, Atom
from deeprankcore.models.contact import AtomicContact, ResidueContact
from deeprankcore.models.graph import Edge, Graph
from deeprankcore.tools.pdb import get_structure
from deeprankcore.models.amino_acid import alanine
from deeprankcore.models.variant import SingleResidueVariant
from deeprankcore.domain.features import edgefeats as Efeat

from typing import List
import logging
import numpy as np
from scipy.spatial import distance_matrix
from deeprankcore.models.structure import Atom
from deeprankcore.models.graph import Graph, Edge
from deeprankcore.models.contact import ResidueContact, AtomicContact
from deeprankcore.domain.features import edgefeats as Efeat
from deeprankcore.domain.forcefield import atomic_forcefield, COULOMB_CONSTANT, EPSILON0, MAX_COVALENT_DISTANCE
from deeprankcore.models.forcefield.vanderwaals import VanderwaalsParam
from deeprankcore.models.error import UnknownAtomError

from deeprankcore.feature.atomic_contact import get_coulomb_potentials, get_lennard_jones_potentials
from tests.feature.test_atomic_contact import _get_atom, _wrap_in_graph

pdb_path = "/home/dbodor/git/DeepRank/deeprank-core/tests/data/pdb/101M/101M.pdb"

pdb = pdb2sql(pdb_path)
try:
    structure = get_structure(pdb, "101m")
finally:
    pdb._close() # pylint: disable=protected-access


In [218]:
RES1 = 0
RES2 = 2

contact = ResidueContact(
    structure.chains[0].residues[RES1], structure.chains[0].residues[RES2]
)
edge = Edge(contact)


In [ ]:
def LJPOT(distances: np.ndarray, atoms: List[Atom],
                                 vanderwaals_parameters: List[VanderwaalsParam]) -> np.ndarray:
    """ Calculate Lennard-Jones potentials, given a distance matrix and a list of atoms with vanderwaals parameters of equal size.

         Warning: there's no distance cutoff here. The radius of influence is assumed to infinite
    """

    # check for the correct data shapes
    atom_count = len(atoms)
    if atom_count != len(vanderwaals_parameters):
        raise ValueError(f"The number of atoms ({atom_count}) does not match the number of vanderwaals parameters ({len(vanderwaals_parameters)})")
    if atom_count != distances.shape[0] or atom_count != distances.shape[1]:
        raise ValueError("Cannot calculate potentials between {} atoms and {} distances" # pylint: disable=consider-using-f-string
                         .format(atom_count, "x".join([str(d) for d in distances.shape])))

    # collect parameters
    sigmas1 = np.empty((atom_count, atom_count))
    sigmas2 = np.empty((atom_count, atom_count))
    epsilons1 = np.empty((atom_count, atom_count))
    epsilons2 = np.empty((atom_count, atom_count))
    for atom1_index in range(atom_count):
        for atom2_index in range(atom_count):
            atom1 = atoms[atom1_index]
            atom2 = atoms[atom2_index]

            # Which parameter we take, depends on whether the contact is intra- or intermolecular.
            if atom1.residue != atom2.residue:

                sigmas1[atom1_index][atom2_index] = vanderwaals_parameters[atom1_index].inter_sigma
                sigmas2[atom1_index][atom2_index] = vanderwaals_parameters[atom2_index].inter_sigma

                epsilons1[atom1_index][atom2_index] = vanderwaals_parameters[atom1_index].inter_epsilon
                epsilons2[atom1_index][atom2_index] = vanderwaals_parameters[atom2_index].inter_epsilon
            else:
                sigmas1[atom1_index][atom2_index] = vanderwaals_parameters[atom1_index].intra_sigma
                sigmas2[atom1_index][atom2_index] = vanderwaals_parameters[atom2_index].intra_sigma

                epsilons1[atom1_index][atom2_index] = vanderwaals_parameters[atom1_index].intra_epsilon
                epsilons2[atom1_index][atom2_index] = vanderwaals_parameters[atom2_index].intra_epsilon

    # calculate potentials
    sigmas = 0.5 * (sigmas1 + sigmas2)
    epsilons = np.sqrt(sigmas1 * sigmas2)
    potentials = 4.0 * epsilons * ((sigmas / distances) ** 12 - (sigmas / distances) ** 6)

    return potentials

In [225]:



def add_features_for_residues(edges): # pylint: disable=too-many-locals
    # get a set of all the atoms involved
    atoms = set([])
    for edge in edges:
        contact = edge.id
        for atom in (contact.residue1.atoms + contact.residue2.atoms):
            atoms.add(atom)
    atoms = list(atoms)

    # get all atomic parameters
    atom_indices = {}
    positions = []
    atom_charges = []
    atom_vanderwaals_parameters = []
    atom_chains = []
    for atom_index, atom in enumerate(atoms):

        try:
            charge = atomic_forcefield.get_charge(atom)
            vanderwaals = atomic_forcefield.get_vanderwaals_parameters(atom)

        except UnknownAtomError:
            print("Ignoring atom %s, because it's unknown to the forcefield", atom)

            # set parameters to zero, so that the potential becomes zero
            charge = 0.0
            vanderwaals = VanderwaalsParam(0.0, 0.0, 0.0, 0.0)

        atom_charges.append(charge)
        atom_vanderwaals_parameters.append(vanderwaals)
        positions.append(atom.position)
        atom_indices[atom] = atom_index
        atom_chains.append(atom.residue.chain.id)


    # calculate the distance matrix for those atoms
    interatomic_distances = distance_matrix(positions, positions, p=2)
    dist_df = pd.DataFrame(columns=atoms, data=interatomic_distances)

    # calculate potentials
    interatomic_electrostatic_potentials = get_coulomb_potentials(interatomic_distances, atom_charges)
    interatomic_vanderwaals_potentials = get_lennard_jones_potentials(interatomic_distances, atoms, atom_vanderwaals_parameters)

    electrostat_df = pd.DataFrame(columns=atoms, data=interatomic_electrostatic_potentials)
    vdw_df = pd.DataFrame(columns=atoms, data=interatomic_vanderwaals_potentials)

    # determine which atoms are close enough to form a covalent bond
    covalent_neighbours = interatomic_distances < MAX_COVALENT_DISTANCE

    # set the edge features
    for _, edge in enumerate(edges):
        contact = edge.id
        edge.features[Efeat.ELECTROSTATIC] = 0
        edge.features[Efeat.VANDERWAALS] = 0

        for atom1 in contact.residue1.atoms:
            for atom2 in contact.residue2.atoms:

                atom1_index = atom_indices[atom1]
                atom2_index = atom_indices[atom2]

                edge.features[Efeat.DISTANCE] = min(edge.features.get(Efeat.DISTANCE, 1e99),
                                                              interatomic_distances[atom_indices[atom1], atom_indices[atom2]])

                edge.features[Efeat.VANDERWAALS] = (edge.features.get(Efeat.VANDERWAALS, 0.0) +
                                                              interatomic_vanderwaals_potentials[atom1_index, atom2_index])

                edge.features[Efeat.ELECTROSTATIC] = (edge.features.get(Efeat.ELECTROSTATIC, 0.0) +
                                                          interatomic_electrostatic_potentials[atom1_index, atom2_index])

                if covalent_neighbours[atom1_index, atom2_index]:
                    edge.features[Efeat.COVALENT] = 1.0
                elif Efeat.COVALENT not in edge.features:
                    edge.features[Efeat.COVALENT] = 0.0

            if atom_chains[atom1_index] == atom_chains[atom2_index]:
                edge.features[Efeat.SAMECHAIN] = 1.0
            else:
                edge.features[Efeat.SAMECHAIN] = 0.0
    
    return dist_df, electrostat_df, vdw_df


dist_df, electrostat_df, vdw_df = add_features_for_residues(_wrap_in_graph(edge).edges)

print('distance', edge.features[Efeat.DISTANCE])
print('vdw', edge.features[Efeat.VANDERWAALS])
print('electrostat', edge.features[Efeat.ELECTROSTATIC])


# original: 1.3296811647910176



distance 4.501408335176893
vdw -28.785803151822083
electrostat 4.495072381413216


/home/dbodor/git/DeepRank/deeprank-core/deeprankcore/feature/atomic_contact.py:30: RuntimeWarning: divide by zero encountered in true_divide
  potentials = np.expand_dims(charges, axis=0) * np.expand_dims(charges, axis=1) \
/home/dbodor/git/DeepRank/deeprank-core/deeprankcore/feature/atomic_contact.py:30: RuntimeWarning: invalid value encountered in true_divide
  potentials = np.expand_dims(charges, axis=0) * np.expand_dims(charges, axis=1) \
/home/dbodor/git/DeepRank/deeprank-core/deeprankcore/feature/atomic_contact.py:79: RuntimeWarning: divide by zero encountered in true_divide
  potentials = 4.0 * epsilons * ((sigmas / distances) ** 12 - (sigmas / distances) ** 6)
/home/dbodor/git/DeepRank/deeprank-core/deeprankcore/feature/atomic_contact.py:79: RuntimeWarning: invalid value encountered in subtract
  potentials = 4.0 * epsilons * ((sigmas / distances) ** 12 - (sigmas / distances) ** 6)


In [167]:
print(dist_df.shape)
print(electrostat_df.shape)
print(vdw_df.shape)

electrostat_df

(15, 15)
(15, 15)
(15, 15)


,101m A 1 O,101m A 1 C,101m A 1 CA,101m A 1 N,101m A 1 CG2,101m A 0 SD,101m A 1 CG1,101m A 0 C,101m A 0 CB,101m A 0 O,101m A 1 CB,101m A 0 N,101m A 0 CE,101m A 0 CG,101m A 0 CA
0,inf,-67.399534,-13.810802,33.740041,0.0,11.301994,-0.0,-20.243632,0.0,17.088136,-0.0,14.849443,-6.591771,-6.543368,-6.669384
1,-67.399534,inf,21.716063,-38.685037,-0.0,-11.564910,0.0,22.877416,-0.0,-20.175215,0.0,-15.592952,6.646915,6.900163,6.916659
2,-13.810802,21.716063,inf,-26.016414,-0.0,-4.876211,0.0,13.664792,-0.0,-12.011679,0.0,-7.762225,2.632461,3.123785,3.493652
3,33.740041,-38.685037,-26.016414,inf,0.0,16.518257,-0.0,-71.173548,0.0,42.116132,-0.0,29.443388,-8.668519,-11.317726,-15.579833
4,0.000000,-0.000000,-0.000000,0.000000,NaN,0.000000,-0.0,-0.000000,0.0,0.000000,-0.0,0.000000,-0.000000,-0.000000,-0.000000
5,11.301994,-11.564910,-4.876211,16.518257,0.0,inf,-0.0,-16.405146,0.0,15.287291,-0.0,19.597651,-20.344555,-20.429011,-7.562291
6,-0.000000,0.000000,0.000000,-0.000000,-0.0,-0.000000,NaN,0.000000,-0.0,-0.000000,0.0,-0.000000,0.000000,0.000000,0.000000
7,-20.243632,22.877416,13.664792,-71.173548,-0.0,-16.405146,0.0,inf,-0.0,-67.624591,0.0,-38.214570,7.908259,12.674061,21.817036
8,0.000000,-0.000000,-0.000000,0.000000,0.0,0.000000,-0.0,-0.000000,NaN,0.000000,-0.0,0.000000,-0.000000,-0.000000,-0.000000
9,17.088136,-20.175215,-12.011679,42.116132,0.0,15.287291,-0.0,-67.624591,0.0,inf,-0.0,34.736863,-7.192777,-11.606819,-13.892780


In [221]:
# redefined distances thingy


def get_residue_features(edges):
    for edge in edges:
        contact = edge.id
        atoms1 = contact.residue1.atoms
        atoms2 = contact.residue2.atoms

        # distance & covalentness
        positions1 = [x.position for x in atoms1]
        positions2 = [x.position for x in atoms2]

        distances = distance_matrix(positions1, positions2)
        distance = np.min(distances)
        covalent = distance < MAX_COVALENT_DISTANCE

        samechain = contact.residue1.chain.id == contact.residue2.chain.id

        # electrostatic
        charges1 = [atomic_forcefield.get_charge(x) for x in atoms1]
        charges2 = [atomic_forcefield.get_charge(x) for x in atoms2]
        potentials = np.expand_dims(charges1, axis=1) * np.expand_dims(charges2, axis=0) \
                 * COULOMB_CONSTANT / (EPSILON0 * distances)
        
        # vdw
        vdw1 = [atomic_forcefield.get_vanderwaals_parameters(x) for x in atoms1]
        vdw2 = [atomic_forcefield.get_vanderwaals_parameters(x) for x in atoms2]
        
        

        print (contact.residue1.chain.id)
        print('distance:', distance)
        print('distances shape:', distances.shape)
        print('covalent:', covalent)
        print('samechain:', samechain)
        print('electrostat:', np.sum(potentials))


get_residue_features(_wrap_in_graph(edge).edges)



A
distance: 4.501408335176893
distances shape: (8, 8)
covalent: False
samechain: True
electrostat: 4.495072381413218


In [237]:
np.zeros((7,8))

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [203]:


A = [.25,.63,.13,.64,.15]
dist = np.array([[.21,.42,.63,.24,.15],
                 [.41,.52,.83,.41,.55],
                 [.11,.62,.73,.4,.25],
                 [.13,.52,.93,.34,.65],
                 [.31,.21,.13,.64,.85],
                    ])

# print(dist.shape)

A0 = np.expand_dims(A, axis=0)
A1 = np.expand_dims(A, axis=1)


ori = A0 * A1 * COULOMB_CONSTANT / (EPSILON0*dist)
new = np.square(A) * COULOMB_CONSTANT / (EPSILON0*dist)

print(A0*A1)
print(np.square(A))

ori_df = pd.DataFrame(data=ori)
new_df = pd.DataFrame(data=new)

# numpy.testing.assert_array_equal (ori,new)

# print(ori_df)
# print(new_df)

assert ori_df.shape == new_df.shape



[[0.0625 0.1575 0.0325 0.16   0.0375]
 [0.1575 0.3969 0.0819 0.4032 0.0945]
 [0.0325 0.0819 0.0169 0.0832 0.0195]
 [0.16   0.4032 0.0832 0.4096 0.096 ]
 [0.0375 0.0945 0.0195 0.096  0.0225]]
[0.0625 0.3969 0.0169 0.4096 0.0225]


In [187]:

def add_features_for_residues(edges): # pylint: disable=too-many-locals
    # get a set of all the atoms involved
    atoms = set([])
    for edge in edges:
        contact = edge.id
        for atom in (contact.residue1.atoms + contact.residue2.atoms):
            atoms.add(atom)
    atoms = list(atoms)


    # get all atomic parameters
    atom_indices = {}
    positions = []
    atom_charges = []
    atom_vanderwaals_parameters = []
    atom_chains = []
    for atom_index, atom in enumerate(atoms):

        try:
            charge = atomic_forcefield.get_charge(atom)
            vanderwaals = atomic_forcefield.get_vanderwaals_parameters(atom)

        except UnknownAtomError:
            _log.warning("Ignoring atom %s, because it's unknown to the forcefield", atom)

            # set parameters to zero, so that the potential becomes zero
            charge = 0.0
            vanderwaals = VanderwaalsParam(0.0, 0.0, 0.0, 0.0)

        atom_charges.append(charge)
        atom_vanderwaals_parameters.append(vanderwaals)
        positions.append(atom.position)
        atom_indices[atom] = atom_index
        atom_chains.append(atom.residue.chain.id)


    # calculate the distance matrix for those atoms
    interatomic_distances = distance_matrix(positions, positions, p=2)
    df = pd.DataFrame(columns=atoms, data=interatomic_distances)

    # calculate potentials from function
    interatomic_electrostatic_potentials = COULOMB_POT(interatomic_distances, atom_charges)
    
    # calculate the potentials here
    potentials = np.expand_dims(atom_charges, axis=0) * np.expand_dims(atom_charges, axis=1) \
                 * COULOMB_CONSTANT / (EPSILON0 * interatomic_distances)
    potentials_sq = np.square(atom_charges) * COULOMB_CONSTANT / (EPSILON0 * interatomic_distances)

    df_pot_ori = pd.DataFrame(columns=atoms, data=potentials)
    df_pot_new = pd.DataFrame(columns=atoms, data=potentials_sq)

    # interatomic_electrostatic_potentials = get_coulomb_potentials(interatomic_distances, atom_charges)
    interatomic_vanderwaals_potentials = get_lennard_jones_potentials(interatomic_distances, atoms, atom_vanderwaals_parameters)

    # determine which atoms are close enough to form a covalent bond
    covalent_neighbours = interatomic_distances < MAX_COVALENT_DISTANCE

    # set the edge features
    for _, edge in enumerate(edges):
        contact = edge.id




        for atom1 in contact.residue1.atoms:
            for atom2 in contact.residue2.atoms:

                atom1_index = atom_indices[atom1]
                atom2_index = atom_indices[atom2]
                # X+=1
                # print(X, 'indices', atom1_index, atom2_index)

                edge.features[Efeat.DISTANCE] = min(edge.features.get(Efeat.DISTANCE, 1e99),
                                                              interatomic_distances[atom_indices[atom1], atom_indices[atom2]])

                edge.features[Efeat.VANDERWAALS] = (edge.features.get(Efeat.VANDERWAALS, 0.0) +
                                                              interatomic_vanderwaals_potentials[atom1_index, atom2_index])

                edge.features[Efeat.ELECTROSTATIC] = (edge.features.get(Efeat.ELECTROSTATIC, 0.0) +
                                                          interatomic_electrostatic_potentials[atom1_index, atom2_index])

                if covalent_neighbours[atom1_index, atom2_index]:
                    edge.features[Efeat.COVALENT] = 1.0
                elif Efeat.COVALENT not in edge.features:
                    edge.features[Efeat.COVALENT] = 0.0

            if atom_chains[atom1_index] == atom_chains[atom2_index]:
                edge.features[Efeat.SAMECHAIN] = 1.0
            else:
                edge.features[Efeat.SAMECHAIN] = 0.0
    
    return interatomic_distances, df_pot_ori, df_pot_new

# check that we can calculate residue contacts
contact = ResidueContact(
    structure.chains[0].residues[0], structure.chains[0].residues[1]
)
edge = Edge(contact)


interatomic_distances, df_pot_ori, df_pot_new = add_features_for_residues(_wrap_in_graph(edge).edges)


# print (edge.features[Efeat.DISTANCE])
# original: 1.3296811647910176



/tmp/ipykernel_21166/246513508.py:14: RuntimeWarning: divide by zero encountered in true_divide
  potentials = np.expand_dims(charges, axis=0) * np.expand_dims(charges, axis=1) \
/tmp/ipykernel_21166/246513508.py:14: RuntimeWarning: invalid value encountered in true_divide
  potentials = np.expand_dims(charges, axis=0) * np.expand_dims(charges, axis=1) \
/tmp/ipykernel_21166/246513508.py:17: RuntimeWarning: divide by zero encountered in true_divide
  potentials_sq = np.square(charges) * COULOMB_CONSTANT / (EPSILON0 * distances)
/tmp/ipykernel_21166/246513508.py:17: RuntimeWarning: invalid value encountered in true_divide
  potentials_sq = np.square(charges) * COULOMB_CONSTANT / (EPSILON0 * distances)


AssertionError: 
Arrays are not equal

Mismatched elements: 144 / 225 (64%)
Max absolute difference: 152.31139239
Max relative difference: 3.85
 x: array([[       inf, -67.399534, -13.810802,  33.740041,   0.      ,
         11.301994,  -0.      , -20.243632,   0.      ,  17.088136,
         -0.      ,  14.849443,  -6.591771,  -6.543368,  -6.669384],...
 y: array([[      inf, 67.399534,  5.524321, 38.463647,  0.      , 10.623875,
         0.      , 20.243632,  0.      , 17.088136,  0.      , 16.928365,
         3.098132,  3.075383,  2.667754],...

In [27]:
df.shape
df

,101m A 1 O,101m A 1 C,101m A 1 CA,101m A 1 N,101m A 1 CG2,101m A 0 SD,101m A 1 CG1,101m A 0 C,101m A 0 CB,101m A 0 O,101m A 1 CB,101m A 0 N,101m A 0 CE,101m A 0 CG,101m A 0 CA
0,0.000000,1.231698,2.404376,2.804920,4.548835,6.904529,3.670643,4.100840,5.167031,4.858101,3.153974,6.373177,5.919119,5.962904,4.978925
1,1.231698,0.000000,1.529115,2.446375,3.869423,6.747562,2.913030,3.628727,5.097405,4.114747,2.518203,6.069288,5.870013,5.654573,4.800925
2,2.404376,1.529115,0.000000,1.455053,2.532858,6.401278,2.530376,2.430067,4.457741,2.764506,1.558869,4.876856,5.928668,4.996179,3.801908
3,2.804920,2.446375,1.455053,0.000000,2.984439,5.385546,3.809494,1.329681,3.134844,2.247075,2.481006,3.664234,5.131202,3.930111,2.429760
4,4.548835,3.869423,2.532858,2.984439,0.000000,7.939645,2.475808,3.219598,5.664228,3.093113,1.522883,4.882507,7.926049,6.246341,4.414654
5,6.904529,6.747562,6.401278,5.385546,7.939645,0.000000,8.906000,4.756736,2.730420,5.104563,7.773226,4.539311,1.802764,1.795311,4.127582
6,3.670643,2.913030,2.530376,3.809494,2.475808,8.906000,0.000000,4.651397,6.913913,4.656596,1.510199,6.871350,8.394155,7.447139,6.026330
7,4.100840,3.628727,2.430067,1.329681,3.219598,4.756736,4.651397,0.000000,2.492032,1.227599,3.308801,2.476493,4.933763,3.078530,1.522038
8,5.167031,5.097405,4.457741,3.134844,5.664228,2.730420,6.913913,2.492032,0.000000,3.284442,5.575325,2.474683,3.257850,1.515570,1.523056
9,4.858101,4.114747,2.764506,2.247075,3.093113,5.104563,4.656596,1.227599,3.284442,0.000000,3.515943,2.724429,5.424535,3.361599,2.390188


In [39]:
def dummy():
    A = [3,3]
    B = [4,4]
    C = [4,4]
    D = [3,3]

    x1 = [1,2]
    x2 = [0]

    y1 = [1]
    y2 = [0]

    return distance_matrix([A,B,x1], [C])

dummy()

array([[1.41421356],
       [0.        ],
       [3.60555128]])